In [ ]:
from collections import defaultdict

import numpy as np
from duckdb.experimental.spark.sql.functions import avg

from util.data_loader import DataLoader


def pearson_coefficient(u:np.ndarray, v:np.ndarray)->float:
    u_diff = u-np.mean(u)
    v_diff = v-np.mean(v)
    분자 = np.dot(u_diff, v_diff)
    분모 = np.sqrt(sum(u_diff**2)) * np.sqrt(sum(v_diff**2))
    if 분모==0:
        return 0
    return 분자/분모

# dataset = DataLoader().main()
# user_movie_matrix = dataset.train.pivot(index="user_id", columns="movie_id", values='rating')
# user_id2index = dict(zip(user_movie_matrix.index, range(len(user_movie_matrix.index))))
# movie_id2index = dict(zip(user_movie_matrix.columns, range(len(user_movie_matrix.columns))))
# movie_rating_predict = dataset.test.copy()

#매트릭스 만들어 , 그리고 매트릭스 인덱스 대응하는 표만들고 테스트 복사한거를 평가 예측에만 사용
dataset = DataLoader().main()
user_movie_matrix = dataset.train.pivot(index="user_id", columns="movie_id", values="rating")
user_id2index = dict(zip(user_movie_matrix.index, range(len(user_movie_matrix.index))))
movie_id2index = dict(zip(user_movie_matrix.columns, range(len(user_movie_matrix.columns))))
pred_test = dataset.test.copy()
pred_love_items = defaultdict(list)
test_users = pred_test.user_id.unique()
for test_user_id in test_users:
    similar_user_id = []
    similarity = []
    avg = []

    for user_id in user_movie_matrix.index:
        if user_id == test_user_id:
            continue
        u1 = user_movie_matrix.loc[test_user_id, :].to_numpy()
        u2 = user_movie_matrix.loc[user_id, :].to_numpy()

        common_df = (~np.isnan(u1) & ~np.isnan(u2))
        if not common_df.any():
            continue
        u1 = u1[common_df]
        u2 = u2[common_df]
        상관계수 = pearson_coefficient(u1,u2)
        if 상관계수 >0:
            similar_user_id.append(user_id)
            similarity.append(상관계수)
            avg.append(np.mean(u2))
    avg_ = np.mean(user_movie_matrix.loc[test_user_id,:].dropna().to_numpy())
    test_movies = pred_test[pred_test["user_id"]==test_user_id].movie_id.values

    pred_test.loc[(pred_test["user_id"]==test_user_id), "rating_pred"]=avg_

    if similar_user_id:
        for movie_id in test_movies:
            r_xy = user_movie_matrix.loc[similar_user_id,movie_id].to_numpy()
            rating_exists = ~np.isnan(r_xy)
            if not rating_exists.any():
                continue
            r_xy = r_xy[rating_exists]
            rho_1x = np.array(similarity)[rating_exists]
            avg_x = np.array(avg)[rating_exists]
            r_hat_1y = avg_+np.dot(rho_1x, (r_xy-avg_x))/rho_1x.sum()

            pred_test.loc[(pred_test["user_id"]==test_user_id)&(pred_test["movie_id"]==movie_id), "rating_pred"]=r_hat_1y
